<a href="https://colab.research.google.com/github/alialdakheel/BlockCipher_exercise/blob/main/BC_cm2_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
  Cipher model 2: SPN cipher
  Use 4 Sboxes each 4x4 and a linear layer (function) 
  to build a 16-bit block cipher
"""
class CM2:
  """
    @ Param:
      sbox: sepcify the sbox to be used (a list)
      lfx: a function specifing the linear layer
      r: number of rounds
  """
  def __init__(self, sbox, lfx, inv_lfx, r):
    self.sbox = sbox
    self.inv_sbox = [sbox.index(i) for i in range(len(sbox))]
    self.lfx = lfx
    self.inv_lfx = inv_lfx
    self.r = r

  def _bytestolist(self, b):
    return [b[0] >> 4, b[0] & 15, b[1] >> 4, b[1] & 15]
  
  def _listtobytes(self, l):
    return bytes([(l[0] << 4) + l[1], (l[2] << 4) + l[3]])

  def _enc_round(self, s, k):
    sub_out = self._listtobytes([self.sbox[w] for w in self._bytestolist(s)])
    lfx_out = self.lfx(sub_out)
    return bytes([w ^ kw for w, kw in zip(lfx_out, k)])

  def _dec_round(self, s, k):
    lfx_out = self.inv_lfx(s)
    sub_out = self._listtobytes([self.inv_sbox[w] for w in self._bytestolist(lfx_out)])
    return bytes([w ^ kw for w, kw in zip(sub_out, k)])

  def enc(self, p, k):
    assert(len(k) == (self.r + 1) * 2)
    sk = k[0:2]
    s = bytes([w ^ kw for w, kw in zip(p, sk)])
    for i in range(self.r):
      rk = k[i*2+2:i*2+4]
      s = self._enc_round(s, rk)
    return s

  def dec(self, c, k):
    assert(len(k) == (self.r + 1) * 2)
    sk = k[-2:]
    s = bytes([w ^ kw for w, kw in zip(c, sk)])
    for i in range(self.r):
      rk = k[-i*2-4:-i*2-2]
      s = self._dec_round(s, rk)
    return s

In [13]:
sbox = [1,2,3,0,4,5,7,6,8,9,11,10,12,13,15,14]
cm2 = CM2(sbox, lambda x: x[::-1], lambda x: x[::-1], 2)

In [14]:
p = 'he'
b = p.encode()

In [15]:
k = 'helloo'.encode()

In [17]:
c = cm2.enc(b, k)

In [18]:
cm2.dec(c, k)

b'he'